<a href="https://colab.research.google.com/github/kusakina0608/ImageDataProcessing/blob/master/day3%20-%20Parking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parking Space Classification

## 실전 프로젝트: 야외 주차장 빈 공간 구분

In [0]:
import cv2
import numpy as np
import glob
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DIR = '/content/drive/My Drive/CNRPark-Patches-150x150/B/free'

files = glob.glob(DIR+os.sep+'*.jpg')

for f in files:
  img = cv2.imread(f)
  
  if img is None:
    print('unable to open', f)
  else:
    h, w, c = img.shape
    if h != 150 or w != 150 or c != 3:
      print('img ', f, 'has wrong dimension: ', img.shape)
      img.resize((150,150,3))
      cv2.imwrite(f.img)

In [0]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

image_datasets = datasets.ImageFolder('/content/drive/My Drive/CNRPark-Patches-150x150/A', transform=transforms.ToTensor())
dataloader = DataLoader(image_datasets, batch_size=1, shuffle=True, num_workers=8, drop_last=True)

In [0]:
for img, label in dataloader:
  print('img: ', type(img), img.size())
  print('label: ', type(label), label.size(), label)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

for img, label in dataloader:
  imgToShow = img.numpy()
  print(imgToShow.shape)
  imgToShow = np.transpose(imgToShow[0], (1,2,0))
  plt.imshow(imgToShow)
  plt.show()

In [11]:
train_size = int(len(image_datasets)*0.8)
test_size = len(image_datasets) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

NameError: ignored

In [9]:
BATCH_SIZE = 64

dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, drop_last=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, drop_last=True)

NameError: ignored

In [0]:
from torch import nn, optim

class UltraSimpleParkingSpaceClassificationNet(nn.Module):
  def __init__(self):
    super(UltraSimpleParkingSpaceClassificationNet, self).__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(3,32,3,padding=1),
        nn.ReLU()
    )
    self.fc_layers = nn.Sequential(
        nn.Linear(150*150*32,100),
        nn.ReLU(),
        nn.Linear(100,2)
    )
  def forward(self, x):
    out = self.layers(x)
    #print('out', out.size())
    out = out.view(BATCH_SIZE, -1)
    #print('out2', out.size())
    out = self.fc_layers(out)
    return out

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = UltraSimpleParkingSpaceClassificationNet().to(device)
model = model.train()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00002)

num_epoch = 10

for i in range(num_epoch):
  for j, [image, label] in enumerate(dataloader):
    x = image.to(device)
    y_ = label.to(device)
    
    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()
    
    if j % 100 == 0:
      print(i, ' ', j, ' loss is: ', loss.data.item())

In [0]:
correct = 0
total = 0
model = model.eval()

with torch.no_grad():
  for image, label in testloader:
    x = image.to(device)
    y_ = label.to(device)
    
    output = model.forward(x)
    _, output_index = torch.max(output, 1)
    
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
  print('Total: {} Correct: {}'.format(total, correct))
  print("Accuracy of Test Data: {}".format(100*correct/total))